In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('PySpark').getOrCreate()
data = [("James","M",60000),("Michael","M",70000),
        ("Robert",None,400000),("Maria","F",500000),
        ("Jen","",None)]

columns = ["name","gender","salary"]
df = spark.createDataFrame(data = data, schema = columns)
df.show()

+-------+------+------+
 name|gender|salary|
+-------+------+------+
 James| M| 60000|
Michael| M| 70000|
 Robert| null|400000|
 Maria| F|500000|
 Jen| | null|
+-------+------+------+

In [0]:
from pyspark.sql.functions import when
df2 = df.withColumn("new_gender", when(df.gender == "M","Male")
                                 .when(df.gender == "F","Female")
                                 .when(df.gender.isNull() ,"")
                                 .otherwise(df.gender))
df2.show()

+-------+------+------+----------+
 name|gender|salary|new_gender|
+-------+------+------+----------+
 James| M| 60000| Male|
Michael| M| 70000| Male|
 Robert| null|400000| |
 Maria| F|500000| Female|
 Jen| | null| |
+-------+------+------+----------+

In [0]:
from pyspark.sql.functions import col
df2=df.select(col("*"),when(df.gender == "M","Male")
                  .when(df.gender == "F","Female")
                  .when(df.gender.isNull() ,"")
                  .otherwise(df.gender).alias("new_gender"))
df2.show()

+-------+------+------+----------+
 name|gender|salary|new_gender|
+-------+------+------+----------+
 James| M| 60000| Male|
Michael| M| 70000| Male|
 Robert| null|400000| |
 Maria| F|500000| Female|
 Jen| | null| |
+-------+------+------+----------+

In [0]:
from pyspark.sql.functions import expr

#Using Case When on withColumn()
df3 = df.withColumn("new_gender", expr("CASE WHEN gender = 'M' THEN 'Male' " + 
               "WHEN gender = 'F' THEN 'Female' WHEN gender IS NULL THEN ''" +
               "ELSE gender END"))
df3.show(truncate=False)

+-------+------+------+----------+
name |gender|salary|new_gender|
+-------+------+------+----------+
James |M |60000 |Male |
Michael|M |70000 |Male |
Robert |null |400000| |
Maria |F |500000|Female |
Jen | |null | |
+-------+------+------+----------+

In [0]:
#Using Case When on select()
df4 = df.select(col("*"), expr("CASE WHEN gender = 'M' THEN 'Male' " +
           "WHEN gender = 'F' THEN 'Female' WHEN gender IS NULL THEN ''" +
           "ELSE gender END").alias("new_gender"))
df4.show(truncate=False)

+-------+------+------+----------+
name |gender|salary|new_gender|
+-------+------+------+----------+
James |M |60000 |Male |
Michael|M |70000 |Male |
Robert |null |400000| |
Maria |F |500000|Female |
Jen | |null | |
+-------+------+------+----------+

In [0]:
df.createOrReplaceTempView("EMP")
spark.sql("select name, CASE WHEN gender = 'M' THEN 'Male' " + 
               "WHEN gender = 'F' THEN 'Female' WHEN gender IS NULL THEN ''" +
              "ELSE gender END as new_gender from EMP").show()

+-------+----------+
 name|new_gender|
+-------+----------+
 James| Male|
Michael| Male|
 Robert| |
 Maria| Female|
 Jen| |
+-------+----------+

In [0]:
#Concatenate columns using || (sql like)
data=[("James","Bond"),("Scott","Varsa")] 
df=spark.createDataFrame(data).toDF("col1","col2") 
df.withColumn("Name",expr(" col1 ||','|| col2")).show()

+-----+-----+-----------+
 col1| col2| Name|
+-----+-----+-----------+
James| Bond| James,Bond|
Scott|Varsa|Scott,Varsa|
+-----+-----+-----------+

In [0]:
from pyspark.sql.functions import expr
data = [("James","M"),("Michael","F"),("Jen","")]
columns = ["name","gender"]
df = spark.createDataFrame(data = data, schema = columns)

#Using CASE WHEN similar to SQL.
from pyspark.sql.functions import expr
df2=df.withColumn("gender", expr("CASE WHEN gender = 'M' THEN 'Male' " +
           "WHEN gender = 'F' THEN 'Female' ELSE 'unknown' END"))
df2.show()

+-------+-------+
 name| gender|
+-------+-------+
 James| Male|
Michael| Female|
 Jen|unknown|
+-------+-------+

In [0]:
from pyspark.sql.functions import expr
data=[("2019-01-23",1),("2019-06-24",2),("2019-09-20",3)] 
df=spark.createDataFrame(data).toDF("date","increment") 

#Add Month value from another column
df.select(df.date,df.increment,
     expr("add_months(date,increment)")
  .alias("inc_date")).show()

+----------+---------+----------+
 date|increment| inc_date|
+----------+---------+----------+
2019-01-23| 1|2019-02-23|
2019-06-24| 2|2019-08-24|
2019-09-20| 3|2019-12-20|
+----------+---------+----------+

In [0]:
# Providing alias using 'as'
from pyspark.sql.functions import expr
df.select(df.date,df.increment,
     expr("""add_months(date,increment) as inc_date""")
  ).show()

+----------+---------+----------+
 date|increment| inc_date|
+----------+---------+----------+
2019-01-23| 1|2019-02-23|
2019-06-24| 2|2019-08-24|
2019-09-20| 3|2019-12-20|
+----------+---------+----------+

In [0]:
# Using Cast() Function
df.select("increment",expr("cast(increment as string) as str_increment")) \
  .printSchema()

root
-- increment: long (nullable = true)
-- str_increment: string (nullable = true)

In [0]:
# Arthemetic operations
df.select(df.date,df.increment,
     expr("increment + 5 as new_increment")
  ).show()

+----------+---------+-------------+
 date|increment|new_increment|
+----------+---------+-------------+
2019-01-23| 1| 6|
2019-06-24| 2| 7|
2019-09-20| 3| 8|
+----------+---------+-------------+

In [0]:
#Use expr()  to filter the rows
from pyspark.sql.functions import expr
data=[(100,2),(200,3000),(500,500)] 
df=spark.createDataFrame(data).toDF("col1","col2") 
df.filter(expr("col1 == col2")).show()

+----+----+
col1|col2|
+----+----+
 500| 500|
+----+----+